In [0]:
EH_SASL = "org.apache.kafka.common.security.plain.PlainLoginModule required username='$ConnectionString' password='Endpoint=sb://kafkaeventhub01.servicebus.windows.net/;SharedAccessKeyName=new;SharedAccessKey=';"
GROUP_ID = "$Default"
requests = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafkaeventhub01.servicebus.windows.net:9093") \
  .option("subscribe", "requests") \
  .option("kafka.sasl.mechanism","PLAIN") \
  .option("kafka.security.protocol","SASL_SSL") \
  .option("kafka.sasl.jaas.config", EH_SASL ) \
  .option("kafka.request.timeout.ms", "60000") \
  .option("kafka.session.timeout.ms", "60000") \
  .option("kafka.group.id", GROUP_ID) \
  .option("failOnDataLoss", "false") \
  .load()



In [0]:
%scala
import org.apache.kafka.common.security.plain.PlainLoginModule
import org.apache.spark.sql.types._
import org.apache.spark.sql.types.StringType
import org.apache.spark.sql.functions.from_json
import org.apache.spark.sql.types.StructField

//Update values as needed
val TOPIC1 = "requests"
val TOPIC2 = "availability"

val BOOTSTRAP_SERVERS = "kafkaeventhub01.servicebus.windows.net:9093"
val EH_SASL = "org.apache.kafka.common.security.plain.PlainLoginModule required username=\"$ConnectionString\" password=\"Endpoint=sb://kafkaeventhub01.servicebus.windows.net/;SharedAccessKeyName=new;SharedAccessKey=\";"
val GROUP_ID = "$Default"
val request_dataSchema = StructType(
  List(
    StructField("id", StringType, true),
    StructField("timestamp", TimestampType, true),
    StructField("request_type", StringType, true),
    StructField("to_loc", StringType, true),
    StructField("from_loc", StringType, true)
   )
)

val availability_dataSchema = StructType(
  List(
    StructField("id", StringType, true),
    StructField("starttime", TimestampType, true),
    StructField("endtime", TimestampType, true),

    StructField("car_type", StringType, true),
    StructField("location", StringType, true),
    StructField("car_id", StringType, true)
   )
)

//Read stream using Spark's Kafka connector
val request_df = spark.readStream
    .format("kafka")
    .option("subscribe", TOPIC1)
    .option("kafka.bootstrap.servers", BOOTSTRAP_SERVERS)
    .option("kafka.sasl.mechanism", "PLAIN")
    .option("kafka.security.protocol", "SASL_SSL")
    .option("kafka.sasl.jaas.config", EH_SASL)
    .option("kafka.request.timeout.ms", "60000")
    .option("kafka.session.timeout.ms", "60000")
    .option("kafka.group.id", GROUP_ID)
    .option("failOnDataLoss", "false")
    .load()
    .select(from_json($"value".cast(StringType), request_dataSchema).as("value"), $"partition")
    .selectExpr("value.id", "value.timestamp", "value.from_loc", "value.to_loc", "value.request_type", "partition")
    .withWatermark("timestamp", "5 seconds")

val availability_df = spark.readStream
    .format("kafka")
    .option("subscribe", TOPIC2)
    .option("kafka.bootstrap.servers", BOOTSTRAP_SERVERS)
    .option("kafka.sasl.mechanism", "PLAIN")
    .option("kafka.security.protocol", "SASL_SSL")
    .option("kafka.sasl.jaas.config", EH_SASL)
    .option("kafka.request.timeout.ms", "60000")
    .option("kafka.session.timeout.ms", "60000")
    .option("kafka.group.id", GROUP_ID)
    .option("failOnDataLoss", "false")
    .load()
    .select(from_json($"value".cast(StringType), availability_dataSchema).as("value"), $"partition")
    .selectExpr("value.id", "value.starttime", "value.endtime", "value.location", "value.car_type", "value.car_id", "partition")
    .withWatermark("starttime", "5 seconds")
request_df.createOrReplaceTempView("requests")
availability_df.createOrReplaceTempView("availability")

import org.apache.kafka.common.security.plain.PlainLoginModule
import org.apache.spark.sql.types._
import org.apache.spark.sql.types.StringType
import org.apache.spark.sql.functions.from_json
import org.apache.spark.sql.types.StructField
TOPIC1: String = requests
TOPIC2: String = availability
BOOTSTRAP_SERVERS: String = kafkaeventhub01.servicebus.windows.net:9093
EH_SASL: String = org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="Endpoint=sb://kafkaeventhub01.servicebus.windows.net/;SharedAccessKeyName=new;SharedAccessKey=rCRM1EPvMROfVvYmm/T9Yolu5cEc065nIPxz/IpFsZ8=";
GROUP_ID: String = $Default
request_dataSchema: org.apache.spark.sql.types.StructType = StructType(StructField(id,StringType,true),StructField(timestamp,TimestampType,true),StructField(request_type,StringType,true),StructField(to_loc,StringType,true),StructField(from_loc,StringType,true))
availability_dataSchema: org.apache.spark.sql.types.StructType = StructType(StructField(id,StringType,true),StructField(starttime,TimestampType,true),StructField(endtime,TimestampType,true),StructField(car_type,StringType,true),StructField(location,StringType,true),StructField(car_id,StringType,true))
request_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: string, timestamp: timestamp ... 4 more fields]
availability_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: string, starttime: timestamp ... 5 more fields]

In [0]:
%scala
display(request_df.selectExpr("value.id", "value.timestamp", "value.from_loc", "value.to_loc", "value.request_type"))

id,timestamp,from_loc,to_loc,request_type
r_loc_4_loc_1_x1664335861.720213,2022-09-27T20:31:01.720+0000,loc_4,loc_1,x
r_loc_4_loc_15_x1664335861.726209,2022-09-27T20:31:01.726+0000,loc_4,loc_15,x
r_loc_4_loc_11_x1664335861.741357,2022-09-27T20:31:01.741+0000,loc_4,loc_11,x
r_loc_4_loc_11_x1664335861.756874,2022-09-27T20:31:01.756+0000,loc_4,loc_11,x
r_loc_4_loc_12_x1664335861.772328,2022-09-27T20:31:01.772+0000,loc_4,loc_12,x
r_loc_4_loc_14_x1664335861.788529,2022-09-27T20:31:01.788+0000,loc_4,loc_14,x
r_loc_4_loc_6_x1664335861.804721,2022-09-27T20:31:01.804+0000,loc_4,loc_6,x
r_loc_4_loc_0_x1664335861.820832,2022-09-27T20:31:01.820+0000,loc_4,loc_0,x
r_loc_4_loc_7_x1664335861.835838,2022-09-27T20:31:01.835+0000,loc_4,loc_7,x
r_loc_4_loc_19_x1664335861.851929,2022-09-27T20:31:01.851+0000,loc_4,loc_19,x


In [0]:
%scala
display(availability_df.selectExpr("value.id", "value.starttime", "value.endtime", "value.location", "value.car_type", "value.car_id"))

id,starttime,endtime,location,car_type,car_id
c_loc_14_x_1664335921.738851,2022-09-27T20:32:01.738+0000,2022-10-02T20:32:01.738+0000,loc_14,x,c_345
c_loc_1_x_1664335921.74419,2022-09-27T20:32:01.744+0000,2022-10-02T20:32:01.744+0000,loc_1,x,c_793
c_loc_6_x_1664335921.759983,2022-09-27T20:32:01.759+0000,2022-10-02T20:32:01.759+0000,loc_6,x,c_322
c_loc_15_x_1664335921.774679,2022-09-27T20:32:01.774+0000,2022-10-02T20:32:01.774+0000,loc_15,x,c_548
c_loc_3_x_1664335921.791147,2022-09-27T20:32:01.791+0000,2022-10-02T20:32:01.791+0000,loc_3,x,c_515
c_loc_8_x_1664335921.806083,2022-09-27T20:32:01.806+0000,2022-10-02T20:32:01.806+0000,loc_8,x,c_766
c_loc_14_x_1664335921.822278,2022-09-27T20:32:01.822+0000,2022-10-02T20:32:01.822+0000,loc_14,x,c_111
c_loc_9_x_1664335921.837474,2022-09-27T20:32:01.837+0000,2022-10-02T20:32:01.837+0000,loc_9,x,c_347
c_loc_0_x_1664335921.856342,2022-09-27T20:32:01.856+0000,2022-10-02T20:32:01.856+0000,loc_0,x,c_837
c_loc_3_x_1664335921.870287,2022-09-27T20:32:01.870+0000,2022-10-02T20:32:01.870+0000,loc_3,x,c_474


In [0]:
%sql select /*+ REPARTITION(requests.partition) */ * from requests, availability where requests.partition = availability.partition and requests.request_type = availability.car_type and requests.from_loc = availability.location and requests.timestamp > availability.starttime - interval 35 seconds and requests.timestamp < availability.starttime + interval 35 seconds 

id,timestamp,from_loc,to_loc,request_type,partition,id,starttime,endtime,location,car_type,car_id,partition
r_loc_7_loc_5_x1664380166.71743,2022-09-28T08:49:26.717+0000,loc_7,loc_5,x,2,c_loc_7_x_1664380138.20964,2022-09-28T08:48:58.209+0000,2022-10-03T08:48:58.209+0000,loc_7,x,c_456,2
r_loc_7_loc_8_x1664380166.733087,2022-09-28T08:49:26.733+0000,loc_7,loc_8,x,2,c_loc_7_x_1664380138.20964,2022-09-28T08:48:58.209+0000,2022-10-03T08:48:58.209+0000,loc_7,x,c_456,2
r_loc_7_loc_3_x1664380166.747819,2022-09-28T08:49:26.747+0000,loc_7,loc_3,x,2,c_loc_7_x_1664380138.20964,2022-09-28T08:48:58.209+0000,2022-10-03T08:48:58.209+0000,loc_7,x,c_456,2
r_loc_7_loc_0_x1664380166.763684,2022-09-28T08:49:26.763+0000,loc_7,loc_0,x,2,c_loc_7_x_1664380138.20964,2022-09-28T08:48:58.209+0000,2022-10-03T08:48:58.209+0000,loc_7,x,c_456,2
r_loc_7_loc_1_x1664380166.779093,2022-09-28T08:49:26.779+0000,loc_7,loc_1,x,2,c_loc_7_x_1664380138.20964,2022-09-28T08:48:58.209+0000,2022-10-03T08:48:58.209+0000,loc_7,x,c_456,2
r_loc_7_loc_17_x1664380166.793921,2022-09-28T08:49:26.793+0000,loc_7,loc_17,x,2,c_loc_7_x_1664380138.20964,2022-09-28T08:48:58.209+0000,2022-10-03T08:48:58.209+0000,loc_7,x,c_456,2
r_loc_7_loc_0_x1664380166.809126,2022-09-28T08:49:26.809+0000,loc_7,loc_0,x,2,c_loc_7_x_1664380138.20964,2022-09-28T08:48:58.209+0000,2022-10-03T08:48:58.209+0000,loc_7,x,c_456,2
r_loc_7_loc_14_x1664380166.824991,2022-09-28T08:49:26.824+0000,loc_7,loc_14,x,2,c_loc_7_x_1664380138.20964,2022-09-28T08:48:58.209+0000,2022-10-03T08:48:58.209+0000,loc_7,x,c_456,2
r_loc_7_loc_11_x1664380166.83987,2022-09-28T08:49:26.839+0000,loc_7,loc_11,x,2,c_loc_7_x_1664380138.20964,2022-09-28T08:48:58.209+0000,2022-10-03T08:48:58.209+0000,loc_7,x,c_456,2
r_loc_7_loc_18_x1664380166.854861,2022-09-28T08:49:26.854+0000,loc_7,loc_18,x,2,c_loc_7_x_1664380138.20964,2022-09-28T08:48:58.209+0000,2022-10-03T08:48:58.209+0000,loc_7,x,c_456,2
